### Portfolio Analysis based on your Risk Aversion level and timeframe
We have 3 levels of Risk Aversion, with level 3 being the most aggressive
We have 3 timeframes for which we will do our Monte Carlo forecasts: 5, 10, and 20 years.

In [11]:
# Import the required libraries and dependencies
#import questionary   #Tried to include this and didn't work
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

In [12]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv()

True

In [13]:
#User inputs 
while True:
    try:
        portfolio_value = input("To begin, please enter the amount to be invested (USD). If you would like to quit, please enter 'Quit': \n")
    
        if portfolio_value == 'Quit':
            print('You have quit the program. Have a nice day.')
            
            break
        
        portfolio_value = float(portfolio_value)
        
        if portfolio_value < 1000000:
            print("Unfortunately, you do not have enough savings to invest in a portfolio. \nThe minimum required is $1,000,000.")
    
        break
    except ValueError:
        print("Please only enter numerical digits.")

    



To begin, please enter the amount to be invested (USD). If you would like to quit, please enter 'Quit': 
 quit


Please only enter numerical digits.


To begin, please enter the amount to be invested (USD). If you would like to quit, please enter 'Quit': 
 Quit


You have quit the program. Have a nice day.


In [4]:
#portfolio_value = 1000000 #we assume client gives us 1mm to start with.

In [24]:
#Get user input for risk tolerance. User must input 1, 2, or 3. If risk_tolerance is not one of these, we prompt user
#to try again.
risk_tolerance = input("Please select your risk tolerance level from 1-3, 3 being the most aggressive: ")
#print(risk_tolerance)
#type if statment here
if int(risk_tolerance) == 1:
    print("Thank you for your response!  You are a conversative investor.")
    tickers = ["JPM","VTEB","PZA","BNDX","GOOGL"]
    #JP Morgan, Vanguard Tax-Exempt Bond Index Fund ETF, Invesco National AMT-Free Municipal Bond ETF, Vanguard Total International Bond Index Fund
    #Google
elif int(risk_tolerance) == 2:
    print("Thank you for your response!  You are an aggressive investor.")
    tickers = ["FB","AAPL","TSLA","RDS.A","GS"]
    #Facebook, Apple, Tesla, Shell Oil, Goldman Sachs
elif int(risk_tolerance) == 3:
    print("Thank you for your response!  You are a very aggressive investor.")
    tickers = ["NFLX","ADBE","AMZN","PYPL","GME"]
    #Netflix, Adobe, Amazon, Paypal, GameStop

else:
    print("I'm sorry, you need to enter 1, 2, or 3 for the application to work.")
#chaim edit- changed the print function to simplify output
risk_tolerance = int(risk_tolerance) #force it to be a number since the result of input() is a string
print(f"The stocks in your portfolio will be {tickers}")


    

Please select your risk tolerance level from 1-3, 3 being the most aggressive:  6


I'm sorry, you need to enter 1, 2, or 3 for the application to work.
The stocks in your portfolio will be ['FB', 'AAPL', 'TSLA', 'RDS.A', 'GS']


In [23]:
time_horizon = input("Please select your time horizon for the forecast: 5, 10, or 20 years?")
if int(time_horizon) == 5:
    print(f"Thank you for your input! Your holding period will be {time_horizon} years")
elif int(time_horizon) == 10:
        print(f"Thank you for your input! Your holding period will be {time_horizon} years")
elif int(time_horizon) == 15:
        print(f"Thank you for your input! Your holding period will be {time_horizon} years")

else:
    print(f"Please select a time horizon that is only 5,10, or 15 years.")
time_horizon = int(time_horizon) #forcing it to be an int
#chaim edit- changed print function to simplify output
#type if statement here if time_horizon is not 5, 10, or 20 we prompt the user to try again

Please select your time horizon for the forecast: 5, 10, or 20 years? 7


Please select a time horizon that is only 5,10, or 15 years.


In [7]:
# Set the variables for the Alpaca API and secret keys. Will be making calls for historical prices.
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Check the values were imported correctly by evaluating the type of each
display(type(alpaca_api_key))
display(type(alpaca_secret_key))

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version="v2")


NoneType

NoneType

ValueError: ('Key ID must be given to access Alpaca trade API', ' (env: APCA_API_KEY_ID)')

In [ ]:
# Set the tickers for both the bond and stock portion of the portfolio

#This setting of tickers will also need to be made a bit more dynamic again dep on what client selects
#We know this would be risk tolerance level 2 based on Nev's list.
#tickers = ["FB","AAPL","TSLA","RDS.A","GS"] Paul Lopez moved this up on 2021-10-29 to the top depending on if client enters 1, 2, or 3 for risk tolerance

# Set timeframe to 1D 
timeframe = "1D"

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you can use a start and end date of 2020-08-07
start_date = pd.Timestamp("2021-10-27", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-10-27", tz="America/New_York").isoformat()
limit_rows = 1000
#We can possibly make this start and end date a bit more dynamic by just pulling today's or yesterday's
#date and feeding in here as a variable.

In [ ]:
# We use the Alpaca get_barset function to get current closing prices for our portfolio of 5 tickers
prices_df = alpaca.get_barset(tickers, timeframe, start=start_date, end=end_date, limit=limit_rows).df

# checking dataframe to make sure everything looks ok for our closes
prices_df.head()


In [ ]:
# Access the closing prices for our 5 tickers from the Alpaca DataFrame
# We also convert the value to a floating point number
#Again will need to be made dynamic depending on if level 1 2 or 3 risk tolerance level is selected
stock1_close_price = float(prices_df[tickers[0]]["close"])
stock2_close_price = float(prices_df[tickers[1]]["close"])
stock3_close_price = float(prices_df[tickers[2]]["close"])
stock4_close_price = float(prices_df[tickers[3]]["close"])
stock5_close_price = float(prices_df[tickers[4]]["close"])

display(stock1_close_price)
display(stock2_close_price)
display(stock3_close_price)
display(stock4_close_price)
display(stock5_close_price)

#fb_close_price = float(prices_df["FB"]["close"])
#aapl_close_price = float(prices_df["AAPL"]["close"])
#tsla_close_price = float(prices_df["TSLA"]["close"])
#rdsa_close_price = float(prices_df["RDS.A"]["close"])
#gs_close_price = float(prices_df["GS"]["close"])

# Print the AGG closing price
#display(fb_close_price)
#display(aapl_close_price)
#display(tsla_close_price)
#display(rdsa_close_price)
#display(gs_close_price)


In [ ]:
# Calculate the current value of the bond portion of the portfolio
#Again will need to be made dynamic depending on if level 1 2 or 3 risk tolerance level is selected
value_per_share = 200000 #we assume 1 million dollar investment, and 200k per share.
stock1_shares = value_per_share / stock1_close_price
stock2_shares = value_per_share / stock2_close_price
stock3_shares = value_per_share / stock3_close_price
stock4_shares = value_per_share / stock4_close_price
stock5_shares = value_per_share / stock5_close_price

# Print how many shares we get of each stock assuming 200k is available for each one.
print(f"With $200,000 you will be able to purchase {stock1_shares:.0f} shares of {tickers[0]}.")
print(f"With $200,000 you will be able to purchase {stock2_shares:.0f} shares of {tickers[1]}.")
print(f"With $200,000 you will be able to purchase {stock3_shares:.0f} shares of {tickers[2]}.")
print(f"With $200,000 you will be able to purchase {stock4_shares:.0f} shares of {tickers[3]}.")
print(f"With $200,000 you will be able to purchase {stock5_shares:.0f} shares of {tickers[4]}.")


In [ ]:
# Set start and end dates of 10 years back from your current date
#We'll always assume 10 years back. Again we can make this dynamic if we want
#and start with yesterday's date and then go back 10 years.
start_date = pd.Timestamp("2011-10-28", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-10-28", tz="America/New_York").isoformat()

limit_rows = 1000

In [ ]:
# Use the Alpaca get_barset function to make the API call to get the 3 years worth of pricing data
# The tickers and timeframe parameters should have been set in Part 1 of this activity 
# The start and end dates should be updated with the information set above
# Remember to add the df property to the end of the call so the response is returned as a DataFrame
prices_df = alpaca.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=limit_rows
).df

# Display both the first and last five rows of the DataFrame
display(prices_df.head(2))
display(prices_df.tail(2))
tickers

In [ ]:
###Here we run a Monte Carlo simulation of 1000 samples and the time frame could be 5, 10, or 20 years depending on the user's response. We always will assume equal 20% weight for each stock in the portfolio.

In [ ]:
# Configure the Monte Carlo simulation to forecast cumulative returns based on the client's input
# The weights we will assume will be split evenly 20 percent for each ticker.
# We will be running 1000 samples.
#This part will only change based on if client selects 5 10 or 20 years to do the forecast.
MC_even_weight = MCSimulation(
    portfolio_data = prices_df,
    weights = [0.2,0.2,0.2,0.2,0.2],
    num_simulation = 1000,
    num_trading_days = 252*time_horizon  #This variable is 5, 10, or 20 depending on client's input above
)

# Review the simulation input data
MC_even_weight.portfolio_data.head()


In [ ]:
# Run the Monte Carlo simulation to forecast client specified number of years of cumulative returns
MC_even_weight.calc_cumulative_return()

#Tweaked monte carlo py file to spit out a message every 200 simulations.

In [10]:
# Visualize the Monte Carlo simulation by creating an overlay line plot
MC_even_weight_line_plot = MC_even_weight.plot_simulation()


NameError: name 'MC_even_weight' is not defined

In [ ]:
# Here we visualize the probability distribution of the Monte Carlo simulation 
# based on the user's timeframe by plotting a histogram
MC_even_weight_distibution_plot = MC_even_weight.plot_distribution()


#### Step 4: Generate the summary statistics for the Monte Carlo simulation.

In [9]:
# Generate summary statistics from the Monte Carlo simulation results
# We save the results as a variable
MC_even_weight_summary_table = MC_even_weight.summarize_cumulative_return()


# Review the Client's Monte Carlo summary statistics
print(MC_even_weight_summary_table)

NameError: name 'MC_even_weight' is not defined

In [8]:
# We now use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes 
#for the current client portfolio
ci_lower_thirty_cumulative_return = round(MC_even_weight_summary_table[8]*portfolio_value,2)
ci_upper_thirty_cumulative_return = round(MC_even_weight_summary_table[9]*portfolio_value,2)

# Print the result of your calculations
print(f"There is a 95% chance that the current value of ${portfolio_value:.2f} in the portfolio with an "
      f"even 20% weight in each stock over the next {time_horizon} years will end within the range of"
      f" ${ci_lower_thirty_cumulative_return} and ${ci_upper_thirty_cumulative_return}.")


NameError: name 'MC_even_weight_summary_table' is not defined